## Chapter 4 . 기본신경망 구현 

### 1) 인공신경망 
    : 입력값 X에 가중치와 편향을 더한 뒤 활성화 함수를 거쳐 결괏값 Y를 만들어 내는 것 

### 2) 활성화함수 
    
   + Sigmoid \
    : 0과 1사이의 값 출력 
   + ReLU \
    : 0보다 작으면 0, 0보다 크면 입력값 그대로 출력
   + tanh \
    : -1 과 1 사이의 값 출력
    
### 3) 역전파 
    : 출력층의 결과의 오차를 신경망을 따라 입력층까지 역으로 전파하여 계산(가중치 갱신)

## 4-1 Classification

#### 분류 : 패턴을 파악해 여러 종류로 구분하는 작업 
### <털과 날개를 기준으로 포유류와 조류를 구분하는 모델 구현>

In [19]:
import tensorflow as tf
import numpy as np

In [20]:
# 1. 데이터 구성 

## [털, 날개]
x_data = np.array([[0, 0], [1, 0], [1, 1], [0, 0], [0, 0], [0, 1]])

## [기타, 포유류, 조류] : 원-핫 인코딩 형태 
y_data = np.array([
    [1, 0, 0],  # 기타
    [0, 1, 0],  # 포유류
    [0, 0, 1],  # 조류
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1]
])

### <특정 X와 레이블 Y와의 관계를 알아내는 모델>
: X, Y에 실측값을 넣어 학습시킨다. 



#### - 가중치와 편향 변수 설정
  * **가중치 변수 W** : [입력층(특징수), 출력층(레이블 수)]로 설정 
  * **편향 변수 b** : 레이블 수로 설정 



#### - 신경망의 구성
  * **Wx + b의 결과를 활성화 함수 ReLU에 적용하면 끝!**

In [21]:
# 2. 신경망 모델 구성 

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# 2차원 신경망[입력층, 출력층] = [2, 3] 
W = tf.Variable(tf.random_uniform([2, 3], -1., 1.))

#출력 개수(레이블 수)인 3으로 설정 
b = tf.Variable(tf.zeros([3]))

#신경망에 가중치 & 편향 적용
L = tf.add(tf.matmul(X, W), b)

# 활성화 함수: ReLU 함수 적용
L = tf.nn.relu(L)

# softmax 함수 적용 : 전체합이 1인 확률로 만들어주는 함수
model = tf.nn.softmax(L)

In [22]:
# 3. 손실 함수 작성 
## 손실함수는 신경망을 최적화하기 위함임 

# Y:실측값 model: 신경망에서 나온 예측값 
## 예측값과 실제값 사이의 확률 분포의 차이를 비용으로 계산 : cross-entropy 함수 사용 
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(model), axis=1))

#최적화: 경사하강법 
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

In [23]:
# 4. 모델 학습 

#세션 초기화 
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# 학습 100번 진행
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    # 10번에 1번씩 손실값 출력 
    if (step + 1) % 10 == 0:
        print(step + 1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))

10 1.0790175
20 1.0721449
30 1.066224
40 1.0619946
50 1.0594577
60 1.0572492
70 1.0552572
80 1.0532871
90 1.0513235
100 1.0493869


In [24]:
# 5. 학습 결과 확인 
# 요소 중 가장 큰 값의 인덱스 찾아줌 :argmax (함수)
prediction = tf.argmax(model, axis=1)
target = tf.argmax(Y, axis=1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

# 6. 정확도 출력 
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

예측값: [0 1 1 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 83.33


### 학습 횟수를 늘려도 정확도가 높여지지 않는다!
<span style="color:red"> 이유: 신경망이 한 층 밖에 안되기 때문에</span> 



## 4-2 Deep Nerual Network
#### 심층 신경망 : 신경망의 레이어를 여러개로 구성 :: 딥러닝 


### <위의 신경망 모델에 가중치와 편향 추가하기>

In [25]:
# 1. 신경망 모델 구성 (데이터는 위의 데이터 사용)

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# 첫번째 가중치: [2, 10] 
W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))

# 두번째 가중치:[10, 3] 
W2 = tf.Variable(tf.random_uniform([10, 3], -1., 1.))

# 편향: 각 레이어의 아웃풋 갯수
## b1 : 히든 레이어(은닉층)의 뉴런 갯수인 10 
b1 = tf.Variable(tf.zeros([10]))

## b2 : 최종 결과값 (분류개수)인 3
b2 = tf.Variable(tf.zeros([3]))

L1 = tf.add(tf.matmul(X, W1), b1)
L1 = tf.nn.relu(L1)

# 두번째 가중치 W2와 편향 b2를 적용 : 최종 3개의 출력값 
model = tf.add(tf.matmul(L1, W2), b2)


In [26]:
# 2. 손실함수 작성

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)

In [27]:
# 3. 모델 학습 

#세션 초기화 
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# 학습 100번 진행
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    # 10번에 1번씩 손실값 출력 
    if (step + 1) % 10 == 0:
        print(step + 1, sess.run(cost, feed_dict={X: x_data, Y: y_data}))
        
# 4. 학습 결과 확인 
# 요소 중 가장 큰 값의 인덱스 찾아줌 :argmax (함수)
prediction = tf.argmax(model, axis=1)
target = tf.argmax(Y, axis=1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

# 5. 정확도 출력 
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

10 0.7081044
20 0.55829155
30 0.4433968
40 0.34239185
50 0.25619677
60 0.18369663
70 0.12782663
80 0.0881943
90 0.061717447
100 0.04436257
예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 100.00
